In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00424647331237793
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 36.71802608610915
200000 58.28329139897721
300000 55.633113454837016
400000 57.92377242412215
500000 55.92207934155473
600000 59.84014572438444
700000 56.152656920115824
800000 55.35603300218554
900000 57.09563756091811
1000000 55.27267055153916
1100000 40.693313688090136
1200000 34.02645597394679
1300000 54.07531854928957
1400000 55.044325192936455
1500000 54.38396346226524
1600000 55.97425761628089
1700000 57.64685131949436
1800000 55.038814492991435
1900000 55.62592607287751
2000000 56.11255032880315
2100000 56.50799130767569
2200000 42.442234311252015
2300000 28.184716127696078
2400000 55.30953471083276
2500000 54.43492295741996
2600000 54.60526743342387
2700000 53.491640332232116
2800000 57.51979290399462
2900000 54.002770745488945
3000000 53.0827151

29800000 53.74993728357413
29900000 49.86504304126244
30000000 55.556606994205794
30100000 54.8719154349115
30200000 53.596270112161335
30300000 59.13748337724711
30400000 55.684932842337034
30500000 59.14962518786254
30600000 24.320394294753108
30700000 57.83499516803916
30800000 56.48979925077551
30900000 48.67755994588369
31000000 42.21353919310234
31100000 36.20815311250815
31200000 51.278506830865815
31300000 54.00214996889239
31400000 46.97090699406838
31500000 51.73191644897536
31600000 45.634562641492515
31700000 49.35833535109849
31800000 48.00370625246574
31900000 19.682473782998464
32000000 52.32690941949443
32100000 43.5582088106412
32200000 50.687133942570526
32300000 46.705024187042966
32400000 49.883121864553175
32500000 52.857777583848595
32600000 47.74933421609326
32700000 50.285535376392325
32800000 31.70281073867203
32900000 49.21786869903225
33000000 52.75664939927083
33100000 48.460846137103026
33200000 47.88793766285662
33300000 31.920231277904367
33400000 50.9144

59900000 44.16034873471154
60000000 48.18958257621194
60100000 47.35385449967384
60200000 50.35139032731299
60300000 44.156116981042494
60400000 46.096104823977335
60500000 23.514978319994892
60600000 47.84898165557645
60700000 47.98190437268342
60800000 45.48543335702358
60900000 50.50813452122858
61000000 46.6868016918471
61100000 25.499859712004145
61200000 50.50445296082318
61300000 48.71998235707259
61400000 56.38208441827874
61500000 48.54904527445769
61600000 52.803089361535534
61700000 51.26377392421268
61800000 49.885922252907186
61900000 51.866008752184044
62000000 33.244203986126145
62100000 35.810930722633564
62200000 47.13649854581353
62300000 44.123306323990874
62400000 46.765738753816656
62500000 43.83065816921652
62600000 42.62896390209406
62700000 25.609020530469646
62800000 44.47319933882011
62900000 46.04150033424013
63000000 44.52170522506606
63100000 47.57031279220637
63200000 46.16662614533697
63300000 21.112561560192706
63400000 51.40944236415821
63500000 47.0570

90000000 51.03530956735936
90100000 52.951091744977624
90200000 52.982935030181
90300000 54.16335883790134
90400000 49.62320740507614
90500000 50.642855708690455
90600000 49.163122620976644
90700000 47.85909838708774
90800000 26.40664376920775
90900000 41.453540445878865
91000000 48.228779020555784
91100000 48.406445983257235
91200000 24.877703422490555
91300000 39.48724065128105
91400000 47.73270189160044
91500000 46.758282252742426
91600000 30.069410231798106
91700000 38.61206019925146
91800000 47.29357962090249
91900000 49.53132159998691
92000000 48.57355322739442
92100000 51.41383994244792
92200000 52.63037696088443
92300000 53.6048665584666
92400000 48.005777914360564
92500000 49.98495740538888
92600000 47.39405885891931
92700000 49.39154998786649
92800000 45.136670072445675
92900000 44.76124275866765
93000000 28.891389715545717
93100000 33.41888968519673
93200000 45.26692132350536
93300000 44.32488159964105
93400000 24.277091037049257
93500000 40.208165823552555
93600000 43.95136

119400000 41.00646238860789
119500000 38.394684514402684
119600000 17.184523747602842
119700000 42.44663860846892
119800000 12.729433671745548
119900000 40.44521896695465
120000000 22.86436678905157
120100000 38.964627768838895
120200000 40.378691492351074
120300000 36.211232894106004
120400000 42.02459525723653
120500000 37.52124784393122
120600000 39.79468392117179
120700000 42.715207735368175
120800000 40.164301026739
120900000 43.53088309500365
121000000 40.061141515795335
121100000 41.71589900526784
121200000 39.30139652586641
121300000 36.62528393228251
121400000 41.66194119962615
121500000 35.01527431520391
121600000 40.31928035726848
121700000 38.17908985634067
121800000 21.201383979126682
121900000 41.357576855657065
122000000 13.03892883005333
122100000 34.94754610669722
122200000 27.447390901621734
122300000 37.94339690415108
122400000 40.53632157412375
122500000 35.1821894757337
122600000 41.56468121896905
122700000 38.174788143199805
122800000 38.37571045304942
122900000 4

...

638900000 1.3906296310786068
639000000 1.5822499424629735
639100000 1.850415655585138
639200000 1.998950414285759
639300000 2.260115352850218
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 3.018279680831904
639500000 1.8754716246559202
639600000 2.0249623407639747
639700000 1.3985092166612187
639800000 2.0578287384932397
639900000 1.6848952200479654
640000000 1.6147500835297512
640100000 1.7206617605954726
640200000 1.5910564743599624
640300000 1.1490760980939745
640400000 0.9923633371367435
640500000 1.372162327627192
640600000 1.2817440155858701
640700000 2.1608765800778924
640800000 0.948191140808575
640900000 1.0746545519813218
641000000 0.8791688413488369
641100000 0.9240259523247467
641200000 0.9212302681629326
641300000 0.9735587281138719
641400000 1.10975341898768
641500000 0.9866931075778186
641600000 1.724083783866614
641700000 2.0463193701094635
641800000 1.2661364810028142
641900000 1.6802697759768803
642000000 1.1698918808458507
642100000 1

665700000 0.5973334116836654
665800000 0.6034394393502898
665900000 0.5875127363384605
666000000 0.5832450680614313
666100000 0.36705294414548545
666200000 0.3456601070392708
666300000 0.3544463506422381
666400000 0.7509560717202505
666500000 1.542611969865503
666600000 1.1491512130077324
666700000 1.5151494856851129
666800000 1.3289311910455048
666900000 0.7507220957455172
667000000 0.8542726616300014
667100000 1.5329042500442305
667200000 1.5320687923660938
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.45386902343846103
667400000 0.30866499394450386
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.4655386255635618
667600000 2.4711952696422093
667700000 2.0984406467174113
667800000 2.556575092361089
667900000 2.7690371710531227
668000000 1.5437193118391652
668100000 0.5967896579440051
668200000 0.5857476593321422
668300000 0.5715054808197061
668400000 0.5793833380269362
668500000 0.5899787584651913
668600000 0.58935

691500000 1.5973558909659944
691600000 1.4719900578650893
691700000 1.2696859088814756
691800000 1.22154800755809
691900000 2.2364945216860037
692000000 2.028661485321343
692100000 1.4543729503029281
692200000 1.8108053177150798
692300000 0.9354358647518853
692400000 1.4450693875350593
692500000 1.2851306920935974
692600000 1.4210731924675026
692700000 2.123930736301308
692800000 0.9326952507253402
692900000 0.6547417893689871
693000000 1.3649413658581944
693100000 1.353237915479141
693200000 1.2440000842283248
693300000 1.9079080103588966
693400000 0.9804232647637672
693500000 0.9656379542211114
693600000 1.9342706400550351
693700000 2.222455103261533
693800000 1.8515722041672706
693900000 0.889539997606698
694000000 0.7670804173898246
694100000 1.624615432007021
694200000 1.7248955299838054
694300000 2.2288295812221106
694400000 1.6339799353311482
694500000 0.9046192749837186
694600000 1.2979335080630976
694700000 1.2717945631236671
694800000 1.3584036794118988
694900000 1.6803885007

718700000 0.47002745418312947
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.1879383476309399
718900000 0.12306148142676354
719000000 0.12155535647002458
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.23534171855239935
719200000 0.34500920706195837
719300000 0.34135534597052725
719400000 0.4888167173262225
719500000 0.39185784838338955
719600000 0.27205873629174626
719700000 0.28005163175446124
719800000 0.27537953614593413
719900000 0.2629752174804032
720000000 0.32296152838279735
720100000 0.1796902557284799
720200000 0.11888360169477663
720300000 0.11926139344798722
720400000 0.11858806135234964
720500000 0.23955334022179245
720600000 0.3017439855604139
720700000 0.48150680256742634
720800000 0.4860362746464014
720900000 0.2917403255358563
721000000 0.26294169570944437
721100000 0.2

739200000 0.1710376762014038
739300000 0.14618005596038303
739400000 0.1768732963363548
739500000 0.1744885266659909
739600000 0.1713940442980177
739700000 0.2715391904751599
739800000 0.2830520690258609
739900000 0.19047630900172524
740000000 0.16855010755601724
740100000 0.15535368388826712
740200000 0.17164335334862538
740300000 0.2675065249279002
740400000 0.1475829021301912
740500000 0.1393889834278345
740600000 0.15959594399333
740700000 0.16021474347205691
740800000 0.15011096377475197
740900000 0.23185499071718652
741000000 0.17050592019795244
741100000 0.12745066731948124
741200000 0.12937405340953467
741300000 0.12836757922638986
741400000 0.15774516878784828
741500000 0.1392434934717059
741600000 0.18054824813222622
741700000 0.2316591847328186
741800000 0.14469690171859861
best so far: 0
type: [2, 1, 1, 5, 13] 130
cases of this type: 3570125
741900000 0.1392065037381768
742000000 0.05240123667448759
742100000 0.05591047983944416
742200000 0.0665355724157287
742300000 0.0400